In [4]:
from astropy.io import fits
import numpy as np
import glob
import os
from pathlib import Path
import re
from datetime import datetime

In [5]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [36]:
def check_need_smear(path):
    status = 0
    parts = path.split("st_")
    search_string = f"st_{parts[1]}"
    file_name = "/Users/hattoriakihiro/Documents/after_graduate2/preprocess/check.txt"

    # ファイル全体を一度に読み込む
    with open(file_name, 'r') as file:
        file_content = file.read()
        if search_string in file_content:
            status = 1

    return status

In [33]:
def bias_cor(DAY):
    result = 0

    B0 = 3.18 * pow(10,2)
    B1 = -4.12 * pow(10,-2)
    B2 = 2.00 * pow(10,-5)
    #calculate bias
    result = B0 + (B1 * DAY) + (B2 * pow(DAY,2))

    return result

In [34]:
#smear correction
def smear_cor(data, NV, NH, tEXP, Isky):
    tVCT = 12 * 1024 * 10 ** -6#μs

    result = np.zeros((NV,NH))
    Ismear = np.zeros((NV,NH))

    for H in range (NH):
        Ismear[:,H] = 0
        for V in range (NV):
            #calculate smear
            Ismear[:,H] += (tVCT / (tVCT + tEXP)) * ((data[V,H] - Isky)/1024)
        #smear correction
        result[:,H] =data[:,H] - Ismear[:,H]
        # print(Ismear,data[:,H] , "result",result[:,H] )

    return result

In [37]:
#scattered light correction
def make_coord(x,y):
    a = np.array([[[x,y]]])
    coordinate = np.tile(a,((2*x)-1,(2*y)-1,1))
    for i in range((2*x)-1):
        for j in range((2*y)-1):
            coordinate[i][j]=[i,j]
    return coordinate

def make_base(x,y):
    a = np.array([[[x-1,y-1]]])
    base = np.tile(a,((2*x)-1,(2*y)-1,1))
    return base

def calc_dist(coord,base):
    a = base-coord
    b = np.sum(np.square(a), axis=2)
    c = np.sqrt(b)
    return c

def scattered_light(xs, ys, filt, data):
    result = np.zeros((header["NAXIS1"],header["NAXIS2"]))

    Ai = {"ul":{"1":12.0,"2":8.0,"3":1.2,"4":1.0,"5":0.8,"6":0.7},
      "b":{"1":10.0,"2":1.5,"3":0.3,"4":0.4,"5":0.4,"6":0.5},
      "v":{"1":10.0,"2":1.5,"3":0.3,"4":0.4,"5":0.4,"6":0.5},
      "w":{"1":10.0,"2":1.5,"3":0.6,"4":0.8,"5":0.7,"6":0.6},
      "x":{"1":9.0,"2":3.5,"3":2.0,"4":2.7,"5":2.2,"6":0.5},
      "p":{"1":10.0,"2":5.0,"3":8.3,"4":4.0,"5":6.4,"6":1.8},
      "zs":{"1":50.0,"2":16.0,"3":6.0,"4":9.0,"5":9.5,"6":4.5}
    }
    filter_u=Ai[filt]
    sigma = [8,16,32,64,110,710]
    coord = make_coord(xs,ys)
    base = make_base(xs,ys)
    r = calc_dist(coord, base)
    fufoc = 0
    for k in range(6):
        num = str(k + 1)
        fufoc = fufoc + ((filter_u[num]*np.power(10.,-4))/(np.sqrt(2*np.pi)*sigma[k]))*np.exp(-np.power(r,2)/(2*np.power(sigma[k],2)))
    for x in range(xs):
        for y in range(ys):
            result[x][y]=np.sum(np.multiply(fufoc[(xs-1)-x:(2*xs-1)-x,(ys-1)-y:(2*ys-1)-y],data))
    return result

In [35]:
#flat correction
def get_flat(filt):
    result = 0
    home_dir = os.path.expanduser("~")
    p = os.path.join(home_dir, "Documents/gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/preflight")
    for f in glob.glob(p+"/flat_"+filt+".fit"):
        result = fits.open(f)
        result = result[0].data
    return result
def flat_cor(filt, data,startH,startV,x_axs,y_axs):
    result = 0
    flat_im = get_flat(filt)
    
    result = data/flat_im[startV:startV+y_axs,startH:startH+x_axs]

    return result

In [38]:
def DN2rad(filt, data):
    scale_factor = {"ul":6.259, "b":1.254, "v":1, "w":0.645, "x":0.6, "p":1.514, "zs":1}
    rad = 3.42 * pow(10,-3) * data
    result = scale_factor[filt] * rad

    return result

In [ ]:
def pre_correction(path):            
    hdulist = fits.open(path)
    hdu = hdulist[0]
    data = hdu.data
    header = hdu.header
    date = header["UTC_0"]
    date = re.split('[-T]',date)
    l_year = 2003
    l_month = 5
    l_day = 9
    l_date = datetime(l_year,l_month,l_day)
    year = int(date[0])
    month = int(date[1])
    day = int(date[2])
    date = datetime(year,month,day)
    days = date - l_date
    DAY = days.days
    xs = header["NAXIS1"]
    ys = header["NAXIS2"]
    startH = header["START_H"]
    startV = header["START_V"]
    filt = header["FILTER_0"]
    tEXP = header["EXP_0"]
    status = check_need_smear(path)
    if(status == 0):
        bias = bias_cor(DAY)
        bias_data = data - bias
        data = smear_cor(bias_data, startV, startH, tEXP, bias)
    flat_data = flat_cor(filt, data, startH, startV, xs, ys)
    scattered_data = scattered_light(xs, ys, filt, flat_data)

    parts = path.split("st_")
    name = f"cor{status}_st_{parts[1]}"

    hdu = fits.PrimaryHDU(scattered_data)
    hdulist = fits.HDUList([hdu])
    hdulist.writeto(name, overwrite=True)


    

    

In [32]:
new_directory = "output_preprocess"
original_directory = "/Users/hattoriakihiro/Documents/after_graduate2/preprocess/output_preprocess"
# ディレクトリを作成（同名のディレクトリが存在する場合、上書き）
new_path = Path(new_directory)
new_path.mkdir(exist_ok=True)

# ディレクトリを格納するリストを作成
directory_list = []

# ベースディレクトリのパスを指定
home_dir = os.path.expanduser("~")
base_directory = os.path.join(home_dir, "Documents/gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/")

# ベースディレクトリ内のすべてのディレクトリを取得
for directory in os.listdir(base_directory):
    directory_path = os.path.join(base_directory, directory)
    if os.path.isdir(directory_path):
        # ディレクトリ内のファイルを取得
        files = os.listdir(directory_path)
        # "b", "v", "w", "p" が各ファイル名に含まれるセットを検査
        if all(char in "".join(files) for char in "bvpw"):
            if "2005" in directory:
                directory_list.append(directory)

print(directory_list)
for subdir in directory_list:
    image_files = sorted(glob.glob(os.path.join(base_directory, subdir, '*.fit')), key = natural_keys)
    print(image_files)

os.chdir(original_directory)
for directory in directory_list:
    target_dir = os.path.join(original_directory, directory)
    target_path = Path(target_dir)
    target_path.mkdir(exist_ok=True)
    os.chdir(target_dir)
    for f in image_files:
        pre_correction(f)
    print(os.getcwd())
    os.chdir("..")
    print(os.getcwd())

['20050928', '20050917', '20050921', '20050919', '20050918', '20051005', '20050920', '20050929', '20051016', '20051011', '20051018', '20050904', '20051019', '20050903', '20051010', '20051017', '20051119', '20050914', '20051009', '20050925', '20050922', '20051006', '20050923', '20050924', '20051008', '20051030', '20050908', '20050906', '20051023', '20051024', '20051025', '20051022', '20051014', '20051013', '20050909', '20051103', '20051104', '20050829']
['/Users/hattoriakihiro/Documents/gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20050928/st_2414830788_v.fit', '/Users/hattoriakihiro/Documents/gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20050928/st_2414838473_v.fit', '/Users/hattoriakihiro/Documents/gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20050928/st_2414846142_v.fit', '/Users/hattoriakihiro/Documents/gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20050928/st_2414853859_v.fit', '/Users/hattoriakihiro/Documents/gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20050928/st_2414861544_v.fit', '/Users/hattoriakihiro/Documents/gra/